In [ ]:
!pip install langchain_community langchain_groq youtube_transcript_api faiss-cpu

In [3]:
from langchain.document_loaders import YoutubeLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain.vectorstores import FAISS
from langchain.chains import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
import textwrap

In [4]:
embeddings = HuggingFaceEmbeddings()

/tmp/ipykernel_31851/1679543760.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
/tmp/ipykernel_31851/1679543760.py:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
/home/aiot/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as

In [5]:
def create_db_from_youtube_video_url(video_url):
  loader = YoutubeLoader.from_youtube_url(video_url)
  transcript = loader.load()

  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 2000, chunk_overlap = 100)
  docs = text_splitter.split_documents(transcript)
  db = FAISS.from_documents(docs, embeddings)
  return db

In [7]:
def get_response_from_query(db, query, k = 4):
  docs = db.similarity_search(query, k = k)
  docs_page_content = " ".join([d.page_content for d in docs])

  chat = ChatGroq(api_key = "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", model = "llama-3.3-70b-versatile", temperature=0)

  template = """You are a helpful assistant that that can answer questions about youtube videos
        based on the video's transcript: {docs}

        Only use the factual information from the transcript to answer the question.

        If you feel like you don't have enough information to answer the question, say "I don't know"."""

  system_message_prompt = SystemMessagePromptTemplate.from_template(template)
  human_template = "Answer the following question: {question}"

  human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

  chat_prompt = ChatPromptTemplate.from_messages(
      [system_message_prompt, human_message_prompt]
  )

  chain = LLMChain(llm = chat, prompt = chat_prompt)

  response = chain.run(question = query, docs = docs_page_content)

  response = response.replace("\n", "")

  return response, docs


In [13]:
video_url = "https://youtu.be/jcleNkRJyVo?si=CghJjAYVn_2H9-RP"
db = create_db_from_youtube_video_url(video_url)
query = "what is this video about?"
response, docs = get_response_from_query(db, query)
print(textwrap.fill(response, width = 70))

This video is about getting started with Arduino, specifically
installing the Arduino IDE and understanding its basic features, as
well as introducing the concept of robotics and embedded systems
engineering.


In [14]:
print(docs)

[Document(id='d85aca10-959e-4d8c-9015-d13c429d06f4', metadata={'source': 'jcleNkRJyVo'}, page_content='button from your desire board and it will just directly connect your board to the a ID here is the library function as uh we are going to start our programming we will talk about the library in our upcoming videos so debugging is the actually uh process of the coding so from here we will uh understand that the how the code will work or there will be a error how how can we debug the error okay so from the search as you already guessing that search will um operate as a search that if you are going to uh use a void the uh that will show that our we are using two void okay so search will operate by default as it is at is plus so here is the basic thing about the AR ID uh here is the serial floater and serial we will talk about it in the upcoming video so thank you very much I hope to see you again bye'), Document(id='a2ede3ba-b66f-4e70-a39c-74372fc819a6', metadata={'source': 'jcleNkRJyVo'

In [ ]:
#Homework to create a gradio/flask/fastapi/react-js application